In [1]:
import findspark
findspark.init()
import pyspark
from operator import add, itemgetter
import json
from string import punctuation
import math
from pyspark.conf import SparkConf
from string import punctuation
#from nltk.stem import LancasterStemmer

# Calculating count of words in all docs combined

In [2]:
sc = pyspark.SparkContext('local[*]',appName="DocClassification")

In [3]:
documents = sc.textFile("/home/vyom/UGA/DSP/Project1/X_train_large.txt")

In [4]:
labels = sc.textFile("/home/vyom/UGA/DSP/Project1/y_train_small.txt")

In [6]:
splt = documents.flatMap(lambda word: word.split())
splt = splt.map(lambda word: word.lower())

#Removing Punctuation
splt = splt.map(lambda word: word.replace("&quot;",""))
splt = splt.map(lambda word: word.replace("&amp;",""))
cleanWords = splt.map(lambda word: word.strip(punctuation))
cleanWords = cleanWords.filter(lambda word:len(word)>2)

#Removing StopWord
stopWordFile = sc.textFile("stopwords.txt")
stopWord = sc.broadcast(stopWordFile.collect())
lessWords = cleanWords.filter(lambda x: x not in stopWord.value)

In [7]:
word = cleanWords.map(lambda word : (word,1))

In [8]:
CountinAllDocs = word.reduceByKey(add)

In [9]:
CountinAllDocs = CountinAllDocs.sortBy(lambda x: x[1],False)

In [10]:
numWords = sc.broadcast(CountinAllDocs.count())
numDocs = sc.broadcast(documents.count())
wordList = sc.broadcast(CountinAllDocs.keys().collect())

# Calculating term frequency in each doc

In [11]:
def stripWord(x):
    tempList =[]
    for word in x:
        word = word.lower()
        word = word.replace("&quot","")
        word = word.replace("&amp","")
        word = word.strip(punctuation)
        if len(word)>2:
            tempList.append(word)
    return tempList

In [12]:
def stripStopWord(x):
    tempList =[]
    for word in x:
        if word not in stopWord.value:
            tempList.append(word)
    return tempList

In [13]:
def countWord(x):
    dictionary={}
    for word in x:
        if word not in dictionary:
            dictionary[word] = 1
        else:
            dictionary[word]+=1
    return dictionary

In [14]:
bagOfWords = documents.map(lambda word: word.split())
bagOfWords = bagOfWords.map(lambda x: stripWord(x))
#bagOfWords = bagOfWords.map(lambda x: stripStopWord(x))
tf = bagOfWords.map(lambda x: countWord(x))

# Calculating idf

In [15]:
def uniques(x):
    tempList=[]
    for word in x:
        if word not in tempList:
            tempList.append(word);
    return tempList

In [16]:
uniqueList = tf.map(lambda x: uniques(x))

In [17]:
def initializeMap(x):
    tempList = []
    for word in x:
        tempList.append((word,1))
    return tempList

In [18]:
occurences = uniqueList.map(lambda x: initializeMap(x)).flatMap(lambda x: x).reduceByKey(add)

In [19]:
idf = occurences.map(lambda x: (x[0],math.log(numDocs.value/float(x[1]))))

In [20]:
idf2 = sc.broadcast(idf.collectAsMap())

# Calculating Tf-idf

In [21]:
def getTfidf(x):
    tempDict={}
    idfDict=idf2.value
    for k,v in x.items():
        tempDict[k] = v*idfDict[k]
    return tempDict

In [22]:
tfidf = tf.map(lambda x: getTfidf(x) )

In [23]:
tfidfSorted = tfidf.map(lambda x: sorted(x.items(), key=itemgetter(1), reverse = True))

# Implementing Naive Bayes

## Calculating word probability

In [39]:
spltLabels = labels.map(lambda word: word.split(","))

In [40]:
def removeUnnecessary(label):
    tempList=[]
    for word in label:
        if 'CAT' in word:
            tempList.append(word)
    return tempList

In [41]:
requiredLabels = spltLabels.map(lambda label: removeUnnecessary(label))

In [42]:
numberOfDocs = sc.broadcast(requiredLabels.flatMap(lambda x: x).count())

In [43]:
labelsToUse = sc.broadcast(requiredLabels.collect())

In [44]:
def dictToList(x):
    tempList=[]
    for k, v in x.items():
        tempList.append((k,v))
    return tempList

In [45]:
def getProbDoc(x,v):
    tempDict={}
    for label in labelsToUse.value[v]:
        tempDict[label] = x
    return tempDict

In [56]:
ProbDocIndex = tfidfSorted.zipWithIndex()

7356

In [34]:
def totalTfidf(x):
    total =0
    for k,v in x.items():
        total = total+v
    return total

In [59]:
ProbDoc = ProbDocIndex.map(lambda x: getProbDoc(x[0],x[1]))

In [123]:
ProbDocIndex.collect()[0][0]

[('0830', 32.09182467968106),
 ('dollar', 19.46545143346878),
 ('sterling', 18.18625103613848),
 ('4,433.3', 17.806543171448425),
 ('poll', 17.482688370283824),
 ('lyons', 14.587667346580222),
 ('sheer', 14.587667346580222),
 ('labour', 14.4198168767047),
 ('overnight', 14.340606367519982),
 ('gmt', 13.774016449032825),
 ('higher', 13.77162576256875),
 ('early', 13.588222779346147),
 ('pound', 13.243905647976216),
 ('index', 12.692604144361503),
 ("labour's", 12.025799655656094),
 ("britain's", 11.949871979688261),
 ('payrolls', 11.815078624340442),
 ('11.7', 11.624458264731791),
 ('stocks', 11.311702218542111),
 ('radical', 11.142134151098016),
 ('majority', 10.984573711993708),
 ('dax', 10.87507136584897),
 ('though', 10.814862657528526),
 ('cac-40', 10.813528048515463),
 ('paris', 10.74078481975741),
 ('ftse', 9.982497160592132),
 ('election', 9.959325830827538),
 ('inflation', 9.826951416100723),
 ('elsewhere', 9.617854047004222),
 ('firmed', 9.427233687395573),
 ('down', 9.4154043

In [66]:
ProbDocList= ProbDoc.map(lambda x : dictToList(x))

In [67]:
def tfidfListToDict(x):
    tempDict={}
    for word in x:
        tempDict[word[0]] = word[1]
    return tempDict

In [68]:
ProbDocList = ProbDocList.flatMap(lambda x: x).reduceByKey(add)
ProbDocDict= ProbDocList.map(lambda x: (x[0],tfidfListToDict(x[1])))

In [97]:
ProbDocDictSorted = ProbDocDict.map(lambda x: (x[0],sorted(x[1].items(), key=itemgetter(1), reverse = False)))

# Above Code for class wise tfidf and stopwords

In [99]:
def getOurStopwords(x):
    tempList=[]
    for word in x:
        if(word[1]<3.0):
            tempList.append(word[0])
    return tempList

In [100]:
ourStopwords = ProbDocDictSorted.map(lambda x: getOurStopwords(x[1]))

In [111]:
toDownload = ourStopwords.flatMap(lambda x: x).distinct()

In [118]:
rd = toDownload.collect()

In [117]:
result = open('ourStopwords.txt', 'w')
for td in rd:
    result.write("%s\n" % td)

In [37]:
labelWC = ProbDocDict.map(lambda x: (x[0],totalTfidf(x[1])))
labelWordCount =sc.broadcast(labelWC.collectAsMap())

In [38]:
def addAllwords(x):
    tempDict=x[1]
    for word in wordList.value:
        if word not in x[1]:
            tempDict[word] = 0
    return (x[0],tempDict)

In [39]:
ProbDocDictAll = ProbDocDict.map(lambda x: addAllwords(x))

In [40]:
def getWordProbability(x):
    tempDict={}
    for k,v in x[1].items():
        tempDict[k]= (v+0.1)/float(0.1*numWords.value + labelWordCount.value[x[0]])
    return (x[0],tempDict)

In [41]:
wordProbability = ProbDocDictAll.map(lambda x: getWordProbability(x))

In [42]:
def getLogProb(x):
    tempDict= {}
    for k,v in x[1].items():
        tempDict[k] = math.log(v)
    return (x[0],tempDict)

In [43]:
logProbability = wordProbability.map(lambda x: getLogProb(x))

## Calculating Class Probability

In [44]:
classList = requiredLabels.flatMap(lambda x: x)
classCount = classList.map(lambda x: (x,1)).reduceByKey(add)

In [45]:
classProbability = classCount.map(lambda x: (x[0],math.log(x[1]/float(numberOfDocs.value))))
classProb = sc.broadcast(classProbability.collectAsMap())

## Testing the probabilities

In [46]:
sums=0
for k,v in wordProbability.collect()[0][1].items():
    sums = sums+v
print(sums)

1.0000044555281526


#  Prediction

In [47]:
testDocuments = sc.textFile("/home/vyom/UGA/DSP/Project1/X_test_small.txt")
testLabels = sc.textFile("y_test_small.txt")

In [48]:
bagOfWordsTest = testDocuments.map(lambda word: word.split())
bagOfWordsTest = bagOfWordsTest.map(lambda x: stripWord(x))
#bagOfWordsTest = bagOfWordsTest.map(lambda x: stripStopWord(x))

In [49]:
testData = sc.broadcast(bagOfWordsTest.collect())

In [61]:
def TestLogProbSum(x):
    tempDict={}
    for i in range(len(testData.value)):
        logSum=0;
        for word in testData.value[i]:
            if word in x[1]:
                logSum=logSum+x[1][word]
            else:
                logSum = logSum+ math.log((0.1/0.1*float(numWords.value)))
        tempDict[i]= logSum
    return (x[0],tempDict)

In [62]:
logProbSum = logProbability.map(lambda x: TestLogProbSum(x))

In [63]:
def getPrediction(x):
    tempDict={}
    for k,v in x[1].items():
        tempDict[k]= v+classProb.value[x[0]]
    return(x[0],tempDict)

In [64]:
prediction = logProbSum.map(lambda x: getPrediction(x))

In [65]:
def dictToTupple(x):
    tempList=[]
    for k, v in x[1].items():
        tempList.append((k,(x[0],v)))
    return tempList

In [66]:
predictComparison = prediction.map(lambda x: dictToTupple(x)).flatMap(lambda x: x).reduceByKey(add)

In [67]:
def getPrediction(x):
    tempList = []
    for i in x:
        if type(i) == float:
            tempList.append(i)
    return x[tempList.index(max(tempList))*2]

In [68]:
predictedLabel = predictComparison.map(lambda x: getPrediction(x[1]))
predicted = predictedLabel.collect()

In [69]:
correctLabel = testLabels.collect()

In [70]:
count = 0
for i in range(len(predicted)):
    if predicted[i] in correctLabel[i]:
        count = count+1
accuracy = count/float(len(predicted))
print(accuracy)

0.3422982885085575


In [60]:
result = open('y_test_small_prediction.txt', 'w')
for labels in predicted:
    result.write("%s\n" % labels)